In [ ]:
import os
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from zipfile import ZipFile
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [ ]:
if not os.path.exists(os.path.join(".","downloaded_reports")):
    os.makedirs(os.path.join(".","downloaded_reports"))
    print("Creating directory to save files")

In [ ]:
#For Citadel
# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
url = 'https://www.citadelsecurities.com/rule-605-606-statements/'
driver.get(url)
time.sleep(3)
report_links = driver.find_elements(By.PARTIAL_LINK_TEXT,"TCDRG")
# Get the report URL
save_directory = "downloaded_reports"
report_path = os.path.join('.',save_directory, 'CitadelSec')
report_urls = {}
for report_link in report_links:
    report_url = report_link.get_attribute("href")
    name = report_link.text
    report_urls[name] = report_url
if not os.path.exists(report_path):
    os.mkdir(report_path)
for name,report_url in report_urls.items():
    driver.get(report_url)
    # final_url = driver.current_url
    # driver.get(final_url)
    file_name = report_url.split("/")[-1]
    this_path = os.path.join(report_path,file_name)
    text_content = driver.find_element(By.TAG_NAME,'body').text
    with open(this_path, 'w', encoding='utf-8') as f:
        f.write(text_content)
    print(f"Downloaded: {report_url} to {report_path}")


In [ ]:
#For Virtu
# Initialize the Chrome WebDriver

#Virtu links need to be located by Month Year format.

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": r"c:\Project\GTS\downloaded_reports\Virtu",
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
    })
driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.virtu.com/about/transparency/rule-605-and-606-reporting/'
driver.get(url)
time.sleep(3)

all_links = driver.find_elements(By.TAG_NAME, 'a')

# Regular expression to match the date pattern
date_pattern = r'(January|February|March|April|May|June|July|August|September|October|November|December) \d{4}'

# Dictionary to store the matching date and corresponding URL
report_urls = {}

# Loop through all links and apply regular expression matching
for link in all_links:
    link_text = link.text
    if re.fullmatch(date_pattern, link_text):
        url = link.get_attribute('href')
        name = url.split("/")[-1]
        report_urls[name]=link.get_attribute('href')

# Print out the found links
# print(report_urls)

# # Get the report URL
save_directory = "downloaded_reports"
report_path = os.path.join('.',save_directory, 'Virtu')

if not os.path.exists(report_path):
    os.mkdir(report_path)
for name,report_url in report_urls.items():
    #this_path = os.path.join(report_path,name)
    driver.get(report_url)
    # final_url = driver.current_url
    # response = requests.get(final_url)
    # with open(this_path, "wb") as f:
    #     f.write(response.content)
    # print(f"Downloaded: {final_url} to {report_path}")

from glob import glob

all_zips = glob(os.path.join(report_path,"*.zip"))
for file in all_zips:
    print(file)
    with ZipFile(os.path.abspath(file), 'r') as zObject:
        name = file.split('\\')[-1].split('.zip')[0]
    # Extracting specific file in the zip
    # into a specific location.
        print(name)
        zObject.extractall(report_path)
        zObject.close()